In [25]:
import pandas as pd
import numpy as np
import re

df = pd.read_excel('/Users/micahbenson/Downloads/mdc_main_data.xlsx')
df = df.drop(columns=['Picture 2021', 'Picture 2022', 'Ticket Number'])

#text formatting
df = df.apply(lambda x: x.str.lower().str.strip() if x.dtype == object else x)
df.columns = df.columns.str.lower().str.replace(" ", "_").str.strip()

#encode school attendance
school_sems = ["fall_semester_2022", "summer_school_2022", "spring_semester_2023"]
for sem in school_sems: 
    clean = df[sem].apply(lambda x: str(x))
    clean = clean.apply(lambda x: "attend" if "attend" in x else x)
    clean = np.where((clean == "attend") | (clean == "yes"), 1, 0)
    df[sem] = clean

#Create col that determines child/adult
df["Child"] = df["relacion"].apply(lambda x: "child" if "child" in str(x) else "adult")

#More things to do... 
#Normalizing dataset
#fams = df.groupby('family_water_number').first()

#start_dict = fams["fecha_de_inicio_agua"].to_dict()
#This method should work if I set the index to the family number and somehow specify it to fill based on row not col... 
#df["fecha_de_inicio_agua"].fillna(start_dict)


grouped = df.groupby('family_water_number')
grouped["fecha_de_inicio_agua"].first()




#lambda x: int(re.search(r'\d+', x).group())




#fam_cols = ["fecha_de_inicio_agua", "select_for_faf", "attended_pic_day_2021"]
#fams = fams[fam_cols]

family_water_number
family 01                 2021-06-04
family 02                 2021-06-04
family 03                 2021-06-27
family 04                 2021-06-27
family 05                 2021-06-04
                             ...    
family 96                        NaT
family 97 - not besitos          NaT
family 98                        NaT
family 99                        NaT
famiy 70                         NaT
Name: fecha_de_inicio_agua, Length: 154, dtype: datetime64[ns]

In [1]:
import pandas as pd
from src import clean

df = pd.read_excel('/Users/micahbenson/Downloads/mdc_main_data.xlsx')
drop = ['Picture 2021', 'Picture 2022', 'Ticket Number', "Relacion", "# of botellas", 
        "Feria de medico 5/21", "Feria de medico 11/21", "4/22 med fair", 
        "Invite april 2023 med fair", "Family never attended a medical fair"]
df = df.drop(columns=drop)
df = clean.text_clean(df)

df = clean.clean_fall_semester_2022(df)
df = clean.clean_spring_semester_2023(df)
df = clean.clean_summer_school_2022(df)
df = clean.clean_super_saturday(df)
df = clean.clean_weight_loss(df)
df = clean.clean_2021_pic(df)
df = clean.clean_dropout(df)
df = clean.is_child(df)



    Family Water Number Unique Family Number Fall Semester 2022   
0             family 01                  100                 no  \
1             family 01                  101                 no   
2             family 01                  102                 no   
3             family 02                  200                 no   
4             family 02                  201                 no   
..                  ...                  ...                ...   
593          family 922                  NaN                 no   
594          family 924                  NaN                 no   
595          family 924                  NaN                 no   
596          family 925                  NaN                 no   
597          family 926                  NaN                 no   

    Summer School 2022 Spring Semester 2023 Adult Or Child Number   
0                   no                   no                     0  \
1                   no                   no              

TypeError: clean_fall_semester_2022() missing 1 required positional argument: 'name'

In [58]:
import pandas as pd

dict = {
    "A": ["12", "2.434",],
    "B": ["12.32 Boy5", "1.232 kg"]
}

df = pd.DataFrame(dict)

df.applymap(lambda x: float(x) if x.replace(".", "").isnumeric() \
              else (float((re.sub(r'[^.\d]', '', x))) if re.sub(r'[^\d]', '', x).isnumeric() else pd.NA))
.applymap(lambda x: re.sub(r'^(.*?)\s.*', r'\1', x))



,A,B
0,12,12.32
1,2.434,1.232


In [8]:
import pandas as pd 
from src import normalize
from src import clean

data = pd.read_excel('/Users/micahbenson/mdc/mdc_nikita.xlsx', sheet_name="mdc_nikita", dtype=str)

people, school, med = normalize.normalize_all(data)

(598, 286)


In [34]:
#find who went to all med fairs: 

counts = (med.index.get_level_values("Individual Id").value_counts())

all = counts[counts == 5]
med_all = med[med.index.get_level_values("Individual Id").isin(all.index)]
med_all[med_all.index.get_level_values("Individual Id") == "401"]
#med_all
#med[med.index.get_level_values("Individual Id") == 5]



#med[med.index.get_level_values("Individual Id") == "300"]
#med.index.get_level_values("Individual Id").value_counts() ==5 

Weight Height   Bmi Blood Sugar    Heart   
Individual Id Date       Family Id                                             
401           04/01/2022 family 04   146.0   56.5  26.6        <NA>  regular  \
              04/01/2023 family 04    65.9  150.0  29.3        93.0  regular   
              11/01/2022 family 04    61.2  150.0  27.2        86.0  regular   
              11/01/2021 family 04  115.74   4.69  25.7        <NA>      NaN   
              05/01/2021 family 04   107.0   56.0  24.0        <NA>  regular   

                                   Lung                 Blood Pressure   
Individual Id Date       Family Id                                       
401           04/01/2022 family 04  cta                            NaN  \
              04/01/2023 family 04  cta  1. 142/87\n2. 132/80\nmo red.   
              11/01/2022 family 04  cta                         128/88   
              11/01/2021 family 04  NaN                            NaN   
              05/01/2021 family 04  cta                   114/98 (104)   

                                                       Su/Drinks/Day    O2   
Individual Id Date       Family Id                                           
401           04/01/2022 family 04                           2l soda  97.0  \
              04/01/2023 family 04  soda and juice a little, 1x soda  99.0   
              11/01/2022 family 04                  3-4 sodas a week  <NA>   
              11/01/2021 family 04                1 cup, 2x per week  <NA>   
              05/01/2021 family 04                        2 cups/day  <NA>   

                                    Pulse  ...    Vaccine Vision Breathing   
Individual Id Date       Family Id         ...                               
401           04/01/2022 family 04  116.0  ...        NaN    NaN       NaN  \
              04/01/2023 family 04   85.0  ...        NaN    NaN       NaN   
              11/01/2022 family 04  121.0  ...        NaN    NaN       NaN   
              11/01/2021 family 04   <NA>  ...        NaN    NaN       NaN   
              05/01/2021 family 04   <NA>  ...  yes - all    NaN       NaN   

                                   Meds Despensed Meds Type And Dose   
Individual Id Date       Family Id                                     
401           04/01/2022 family 04            NaN                NaN  \
              04/01/2023 family 04            NaN                NaN   
              11/01/2022 family 04            NaN                NaN   
              11/01/2021 family 04            NaN                NaN   
              05/01/2021 family 04            NaN                NaN   

                                                                           Follow Up   
Individual Id Date       Family Id                                                     
401           04/01/2022 family 04                                               NaN  \
              04/01/2023 family 04                                               NaN   
              11/01/2022 family 04                                               NaN   
              11/01/2021 family 04                                               NaN   
              05/01/2021 family 04  1. no more soda 2. no problem hearing and seeing   

                                   Baby Cavities Adult Cavities   
Individual Id Date       Family Id                                
401           04/01/2022 family 04           NaN            NaN  \
              04/01/2023 family 04           NaN            NaN   
              11/01/2022 family 04           NaN            NaN   
              11/01/2021 family 04           NaN            NaN   
              05/01/2021 family 04             4              3   

                                   May 2021 Total Cavities   
Individual Id Date       Family Id                           
401           04/01/2022 family 04                     NaN  \
              04/01/2023 family 04                     NaN   
              11/01/20